In [17]:
import csv
import re
import os

In [18]:
def extract_seizure_info(file_path):
    seizures = []
    print(f"正在读取文件：{file_path}")

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 匹配每一个文件块，从 File Name 到下一个 File Name 或结尾
    file_blocks = re.findall(r"File Name:\s*(.*?)\n(.*?)(?=\nFile Name:|\Z)", content, re.DOTALL)

    for filename, block in file_blocks:
        num_seizures_match = re.search(r"Number of Seizures in File:\s*(\d+)", block)
        if not num_seizures_match:
            print(f"警告：在 {filename} 中找不到 Number of Seizures in File")
            continue

        num_seizures = int(num_seizures_match.group(1))
        if num_seizures == 0:
            print(f"注意：{filename} 中没有癫痫发作记录")
            continue

        print(f"{filename} 中有 {num_seizures} 条癫痫记录")

        # 提取每一个 seizure 的信息
        for i in range(1, num_seizures + 1):
            start_match = re.search(rf"Seizure\s*(?:{i}\s*)?Start\s*Time:\s*(\d+)", block, re.IGNORECASE)
            end_match = re.search(rf"Seizure\s*(?:{i}\s*)?End\s*Time:\s*(\d+)", block, re.IGNORECASE)

            if start_match and end_match:
                seizures.append({
                    "File Name": filename.strip(),
                    "Seizure Start Time": int(start_match.group(1)),
                    "Seizure End Time": int(end_match.group(1))
                })
                print(f"成功提取：{filename}, Seizure {i} - Start: {start_match.group(1)}, End: {end_match.group(1)}")
            else:
                print(f"警告：未能提取 {filename} 中的 Seizure {i} 时间信息")
                print(f"检查 {filename} 的具体内容:")
                print(block)  # 打印相关块的内容以便进一步分析

    return seizures

In [19]:
def save_to_csv(seizure_files, csv_path):
    write_header = not os.path.exists(csv_path) or os.path.getsize(csv_path) == 0

    with open(csv_path, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if write_header:
            writer.writerow(["File Name", "Seizure Start Time (seconds)", "Seizure End Time (seconds)"])
        for seizure_file in seizure_files:
            writer.writerow([seizure_file["File Name"], seizure_file["Seizure Start Time"], seizure_file["Seizure End Time"]])

In [20]:
# 主程序入口
if __name__ == "__main__":
    root_path = "F:/CHB_MIT/chb-mit-scalp-eeg-database-1.0.0/"  # 数据集根目录
    output_csv = "seizure_info.csv"  # 输出CSV文件名

    for i in range(1, 25):  # 处理 chb01 到 chb24
        subject = f"chb{i:02d}"  # 格式化为两位数，如 chb01, chb02, ..., chb24
        txt_path = os.path.join(subject, f"{subject}-summary.txt")  # 构造相对路径
        full_txt_path = os.path.join(root_path, txt_path)  # 完整路径

        if os.path.exists(full_txt_path):
            print(f"✅ 正在处理文件：{full_txt_path}")
            seizure_data = extract_seizure_info(full_txt_path)
            save_to_csv(seizure_data, output_csv)
        else:
            print(f"❌ 警告：未找到文件：{full_txt_path}")

    print(f"\n🎉 所有数据已成功保存至：{output_csv}")

✅ 正在处理文件：F:/CHB_MIT/chb-mit-scalp-eeg-database-1.0.0/chb01\chb01-summary.txt
正在读取文件：F:/CHB_MIT/chb-mit-scalp-eeg-database-1.0.0/chb01\chb01-summary.txt
注意：chb01_01.edf 中没有癫痫发作记录
注意：chb01_02.edf 中没有癫痫发作记录
chb01_03.edf 中有 1 条癫痫记录
成功提取：chb01_03.edf, Seizure 1 - Start: 2996, End: 3036
chb01_04.edf 中有 1 条癫痫记录
成功提取：chb01_04.edf, Seizure 1 - Start: 1467, End: 1494
注意：chb01_05.edf 中没有癫痫发作记录
注意：chb01_06.edf 中没有癫痫发作记录
注意：chb01_07.edf 中没有癫痫发作记录
注意：chb01_08.edf 中没有癫痫发作记录
注意：chb01_09.edf 中没有癫痫发作记录
注意：chb01_10.edf 中没有癫痫发作记录
注意：chb01_11.edf 中没有癫痫发作记录
注意：chb01_12.edf 中没有癫痫发作记录
注意：chb01_13.edf 中没有癫痫发作记录
注意：chb01_14.edf 中没有癫痫发作记录
chb01_15.edf 中有 1 条癫痫记录
成功提取：chb01_15.edf, Seizure 1 - Start: 1732, End: 1772
chb01_16.edf 中有 1 条癫痫记录
成功提取：chb01_16.edf, Seizure 1 - Start: 1015, End: 1066
注意：chb01_17.edf 中没有癫痫发作记录
chb01_18.edf 中有 1 条癫痫记录
成功提取：chb01_18.edf, Seizure 1 - Start: 1720, End: 1810
注意：chb01_19.edf 中没有癫痫发作记录
注意：chb01_20.edf 中没有癫痫发作记录
chb01_21.edf 中有 1 条癫痫记录
成功提取：chb01_21.edf, Seizure 1 - Start: 327, E

In [2]:
import pandas as pd
df = pd.read_csv('seizure_info.csv')
print(df.head())

      File Name  Seizure Start Time (seconds)  Seizure End Time (seconds)
0  chb01_03.edf                          2996                        3036
1  chb01_04.edf                          1467                        1494
2  chb01_15.edf                          1732                        1772
3  chb01_16.edf                          1015                        1066
4  chb01_18.edf                          1720                        1810


In [6]:
total_s = (df['Seizure End Time (seconds)'] - df['Seizure Start Time (seconds)']).sum()
print(f"总时长为: {total_s} 秒")
total_m = total_s/60
print(f"= {total_m} 分")
total_h = total_m/60
print(f"= {total_h} 小时")

总时长为: 11562 秒
= 192.7 分
= 3.2116666666666664 小时
